In [31]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense , Dropout, InputLayer
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split




In [32]:
data=pd.read_excel('clinical_radiomics_imported_from_tsv.xlsx')
data_to_pca=data.loc[:,'original_shape_Elongation':'original_ngtdm_Strength']
target=data.loc[:,'MFS_cens']

In [33]:
scaler=StandardScaler()
data_scaled=scaler.fit_transform(data_to_pca)



In [42]:
pca=PCA(n_components=10)
pca.fit(data_scaled)
data_pca=pca.transform(data_scaled)
data_pca=pd.DataFrame(data_pca)


In [35]:
print(pca.explained_variance_ratio_)

[0.4153947  0.15220385 0.11630564 0.07561596 0.05064658 0.03913676
 0.02630412 0.01690465 0.01379928 0.01229   ]


In [43]:
x_train, x_test,y_train,y_test=train_test_split(data_pca, target, test_size=0.2, random_state=20)



In [87]:
model=Sequential()
model.add(InputLayer(input_shape=(10,)))
# model.add(Dropout(0.7))

# model.add(Dense(20,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='relu'))
model.compile(loss='binary_crossentropy',
               optimizer='adam', #RMSprop(learning_rate=0.001) #SGD(lr=0.01, momentum=0.9)
               metrics=['accuracy'])

In [88]:
kfold=KFold(n_splits=5, shuffle=True,random_state=10)
scores=[]

In [89]:
for train_index, test_index in kfold.split(x_train):

    x_train_model,x_test_model=x_train.iloc[train_index],x_train.iloc[test_index]
    y_train_model,y_test_model=y_train.iloc[train_index],y_train.iloc[test_index]
    for i in range(10):
        model.fit(x_train_model,y_train_model,batch_size=8,epochs=1000,verbose=0) #,callbacks=[tensorboard_callback] #<- jak sie doda do modelu to pokazuje statystyki ale wolno się trenuje 
        score=model.evaluate(x_test,y_test,verbose=1)
    scores.append(score[1])
    print(score)
print('Accuracy: %.2f%% (+/- %.2f%%)' % (np.mean(scores)*100, np.std(scores)*100))

1/1 [==============================] - 0s 21ms/step - loss: 7.1214 - accuracy: 0.4444
[7.1213836669921875, 0.4444444477558136]
1/1 [==============================] - 0s 20ms/step - loss: 5.9431 - accuracy: 0.4815


KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

1/1 - 0s - loss: 4.5248 - accuracy: 0.7037 - 17ms/epoch - 17ms/step
Test accuracy: 0.7037037014961243
